In [1]:
from tqdm import tqdm
import pandas as pd
import Classifier as cls

In [23]:
name = 'SK하이닉스'
day = 60
sizes = [0.3, 0.2, 0.1, 0.05]

stock_code = pd.read_csv('./KOSPI_200.csv', dtype={'종목코드': str, '종목명': str})[['종목명', '종목코드']]
xgb_result_df = pd.DataFrame(columns=['NAME', 'SIZE', 'ACC', 'n_estimators', 'eta', 'min_child_weight', 'max_depth', 'colsample_bytree', 'subsample'])

print(f'Start {name} Modeling!!')
data, code = cls.load_stocks_data(f'{name}', stock_code)

for size in sizes:
    stocks = cls.Stocks(data)
    stocks.preprocessing()
    sign_data = stocks.stocksign(stocks.data, day)
    parameter = stocks.modeling(sign_data, code, day, size)

    xgb_result_df = xgb_result_df.append({'NAME':name,
                                          'SIZE':size,
                                          'ACC':parameter[0]['ACC'],
                                          'n_estimators':parameter[0]['n_estimators'],
                                          'eta':parameter[0]['eta'],
                                          'min_child_weight':parameter[0]['min_child_weight'],
                                          'max_depth':parameter[0]['max_depth'],
                                          'colsample_bytree':parameter[0]['colsample_bytree'],
                                          'subsample':parameter[0]['subsample']}, ignore_index=True)

print('Finish!!!!!')

Start SK하이닉스 Modeling!!
TODAY: 2021-08-22
(6000, 6)
Preprocessing Done!!
StockSign Done!!
000660_60_Modeling Finish!!
Preprocessing Done!!
StockSign Done!!
000660_60_Modeling Finish!!
Preprocessing Done!!
StockSign Done!!
000660_60_Modeling Finish!!
Preprocessing Done!!
StockSign Done!!
000660_60_Modeling Finish!!
Finish!!!!!


In [24]:
xgb_result_df

,NAME,SIZE,ACC,n_estimators,eta,min_child_weight,max_depth,colsample_bytree,subsample
0,SK하이닉스,0.30,0.69,1000.00,0.30,2.00,5.00,0.80,0.80
1,SK하이닉스,0.20,0.56,1000.00,0.30,1.00,3.00,0.80,0.80
2,SK하이닉스,0.10,0.67,1000.00,0.30,7.00,3.00,0.90,0.80
3,SK하이닉스,0.05,0.51,1000.00,0.10,6.00,3.00,0.90,0.80


In [22]:
xgb_120_kakao = xgb_result_df.copy()

In [19]:
xgb_60_kakao = xgb_result_df.copy()

In [16]:
xgb_120_lg = xgb_result_df.copy()

In [13]:
xgb_60_lg = xgb_result_df.copy()

In [10]:
xgb_120 = xgb_result_df.copy()

In [6]:
xgb_60 = xgb_result_df.copy()

In [ ]:
data = data.iloc[:-day, :]

X = data.drop(['High', 'Low', 'Open', 'Close', 'Volume', 'Change', 'up', 'down',
                'ma5', 'ma20', 'ma60', 'ma120', 'target'], axis=1)
y = data[['target']]

# 최대한 최근의 데이터를 학습하기 위해 뒷부분의 0.05%만 test 데이터로 활용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=False)

# 스케일링 적용하여 모든 변수 수치 정규화
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)